# 01 - Business Understanding


Using Data Mining the aim is to idenity and predict the factors 
which affect the use of bike-sharing rental services 

# 02 -Data Exploration 

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/24 07:15:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df = spark.read.csv('train.csv')

df_pd = pd.read_csv('train.csv')

In [ ]:
df.show()

df.columns 

In [ ]:
# rename columns 
df = df.withColumnRenamed('_c0', 'datetime')
df = df.withColumnRenamed('_c1', 'season')
df = df.withColumnRenamed('_c2', 'holiday')
df = df.withColumnRenamed('_c3', 'workday')
df = df.withColumnRenamed('_c4', 'weather')
df = df.withColumnRenamed('_c5', 'temp')
df = df.withColumnRenamed('_c6', 'feels_temp')
df = df.withColumnRenamed('_c7', 'humidity')
df = df.withColumnRenamed('_c8', 'windspeed')
df = df.withColumnRenamed('_c9', 'casual')
df = df.withColumnRenamed('_c10', 'registered')
df = df.withColumnRenamed('_c11', 'count')

df.show()



In [ ]:
#Filter out initial row as it is column lable 
df = df.filter(df.datetime!='datetime')

df.show()

df.columns

In [ ]:
#Dataframe Components Summary 

df.select("datetime", "season", 'holiday', 'workday').summary().show()
df.select('weather',"temp", "feels_temp", 'humidity', 'windspeed').summary().show()
df.select("casual", "registered", 'count',).summary().show()

In [ ]:
df.printSchema()

df_pd.info()


In [ ]:
#Check if there are any null or missing values
from pyspark.sql.functions import col

null_counts = df.select([col(column).isNull().alias(column) for column in df.columns]).toPandas().sum()
print("Null value counts in DataFrame:")
print(null_counts)

In [ ]:
df.select('count',).summary().show()
df_pd['count'].plot.hist()

In [ ]:
df_pd['season'].value_counts().plot.pie(autopct='%1.0f%%')

In [ ]:
df_pd['weather'].value_counts().plot.pie(autopct='%1.0f%%')

# 03 - Data Preparation

In [ ]:
# Import in the relevant types.
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)
df.show() 

In [ ]:
#Split datetime varaible into Year, Month, Day and Hour
from pyspark.sql.functions import split

split_col = split(df['datetime'], '-')

df = df.withColumn('Year', split_col.getItem(0))
df = df.withColumn('Month', split_col.getItem(1))
df = df.withColumn('Day_Time', split_col.getItem(2))


split_col = split(df['Day_Time'], '\s+')
df = df.withColumn('Day', split_col.getItem(0))
df = df.withColumn('Time', split_col.getItem(1))

split_col = split(df['Time'], ':')
df = df.withColumn('Hour', split_col.getItem(0))

df = df.drop('datetime')
df = df.drop('Day_Time')
df = df.drop('Time')
df.show() 


In [ ]:
#Reformt Schema
from pyspark.sql.types import StringType, IntegerType, FloatType, BooleanType

df = df.withColumn("season" ,df["season"].cast(IntegerType()))
df = df.withColumn("holiday" ,df["holiday"].cast(BooleanType()))
df = df.withColumn("workday" ,df["workday"].cast(BooleanType()))
df = df.withColumn("weather" ,df["weather"].cast(IntegerType()))
df = df.withColumn("temp" ,df["temp"].cast(FloatType()))
df = df.withColumn("feels_temp" ,df["feels_temp"].cast(FloatType()))
df = df.withColumn("humidity" ,df["humidity"].cast(IntegerType()))
df = df.withColumn("windspeed" ,df["windspeed"].cast(FloatType()))
df = df.withColumn("casual" ,df["casual"].cast(IntegerType()))
df = df.withColumn("registered" ,df["registered"].cast(IntegerType()))
df = df.withColumn("count" ,df["count"].cast(IntegerType()))
df = df.withColumn("Year" ,df["Year"].cast(IntegerType()))
df = df.withColumn("Month" ,df["Month"].cast(IntegerType()))
df = df.withColumn("Day" ,df["Day"].cast(IntegerType()))
df = df.withColumn("Hour" ,df["Hour"].cast(IntegerType()))


df.printSchema()

In [ ]:
df.show()

In [ ]:
# Reformat Season and Weather Variables 
from pyspark.sql.functions import when

df_reformated = df.select("*") 

df_reformated = df_reformated.withColumn("season",
                   when(df_reformated["season"] == "1", "Spring")
                   .when(df_reformated["season"] == "2", "Summer")
                   .when(df_reformated["season"] == "3", "Autumn")
                   .when(df_reformated["season"] == "4", "Winter")
                   .otherwise(df_reformated["season"]))

df_reformated = df_reformated.withColumn("weather",
                   when(df_reformated["weather"] == "1", "Sunny")
                   .when(df_reformated["weather"] == "2", "Cloudy")
                   .when(df_reformated["weather"] == "3", "Lite R/S")
                   .when(df_reformated["weather"] == "4", "Severe Con")
                   .otherwise(df_reformated["weather"]))

df.show()
df_reformated.show()


In [ ]:
df.select("season", "holiday", 'workday').summary().show()
df.select('weather',"temp", "feels_temp", 'humidity', 'windspeed').summary().show()
df.select("casual", "registered", 'count',).summary().show()
df.select("Year", "Month", 'Day','Hour').summary().show()

In [ ]:
#Data Preparation

#dealing with outliers in humidity and windspeed variables 

from pyspark.sql.functions import col, mean, stddev

# Calculate mean and std deviation for the 'humidity' column
stats = df.select(mean(col("humidity")).alias('mean_humidity'), 
                  stddev(col("humidity")).alias('stddev_humidity')).collect()

# Extract mean and std deviation values
mean_val = stats[0]['mean_humidity']
stddev_val = stats[0]['stddev_humidity']

# Define threshold for outliers (e.g., 3 standard deviations)
threshold = 3

# Filter out outliers based on the Z-score
df = df.filter((col("humidity") > mean_val - threshold * stddev_val) & 
                            (col("humidity") < mean_val + threshold * stddev_val))


stats = df.select(mean(col("windspeed")).alias('mean_windspeed'), 
                    stddev(col("windspeed")).alias('stddev_windspeed')).collect()

# Extract mean and std deviation values
mean_val = stats[0]['mean_windspeed']
stddev_val = stats[0]['stddev_windspeed']

# Define threshold for outliers
threshold = 3.0  # Example threshold as a float

# Filter out outliers based on the Z-score using the threshold
df = df.filter((col("windspeed") > mean_val - threshold * stddev_val) & 
                            (col("windspeed") < mean_val + threshold * stddev_val))


# Show the filtered data
df.show()



In [ ]:
df.select("season", "holiday", 'workday').summary().show()
df.select('weather',"temp", "feels_temp", 'humidity', 'windspeed').summary().show()
df.select("casual", "registered", 'count',).summary().show()
df.select("Year", "Month", 'Day','Hour').summary().show()

# 04 - Data Transformation


In [ ]:
#Data Reduction
#Convert dataframe to Pandas to view correlation matirx
df_pd = df.toPandas()

correlation_matrix = df_pd.corr()

# Print correlation matrix
print("Correlation Matrix:")
print(correlation_matrix['count'])

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
#Drop Varaibles that are seen as to not have a high correlations with target variable "count"
df.show()

df = df.drop('holiday', 'workday', 'weather', 'humidity', 'Day')
df_reformated = df_reformated.drop('holiday', 'workday', 'weather', 'humidity', 'Day')

df.show()
df_reformated.show()


# 05 - Data Mining Methods

### Classification Methods 
- Logistic regression

### Clustering Methods 
- K Means Clustering

### Regression Methods
- Linear Regression

#### Tree Methods
- Single Decision Tree
- Random Forest 
- Gradient Boosted ree Classifer 

# 06 - Data Mining Method Analysis


In [ ]:
# K-Means Clustering 



# 07 - Data Mining


### K-MEANS CLUSTERING
#### PySpark

In [ ]:
from pyspark.ml.clustering import KMeans
df.head(1)

df.printSchema()

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import IntegerType

df = df.withColumn("count", df["count"].cast(IntegerType()))

df.printSchema()

In [ ]:
feat_cols = ['season', 'temp', 'feels_temp', 'windspeed', 'casual', 'registered', 'count', 'Year', 'Month', 'Hour']

vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

final_data = vec_assembler.transform(df)

from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
final_data

In [ ]:
scalerModel = scaler.fit(final_data)
cluster_final_data = scalerModel.transform(final_data)

### Train and Evaluate Model

In [ ]:
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [ ]:
model3 = kmeans3.fit(cluster_final_data)
model2 = kmeans2.fit(cluster_final_data)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Make predictions
predictions3 = model3.transform(cluster_final_data)
predictions2 = model2.transform(cluster_final_data)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette3 = evaluator.evaluate(predictions3)
print("With k=3 Silhouette with squared euclidean distance = " + str(silhouette3))

silhouette2 = evaluator.evaluate(predictions2)
print("With k=2 Silhouette with squared euclidean distance = " + str(silhouette2))

In [ ]:
#RESULTS 
centers3 = model3.clusterCenters()
print("Cluster Centers for k=3:")
for center in centers3:
    print(center)

centers2 = model2.clusterCenters()
print("Cluster Centers for k=2:")
for center in centers2:
    print(center)

In [ ]:
# Evaluate clustering by computing Within Set Sum of Squared Errors.
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
    predictions = model.transform(cluster_final_data)
    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions)
    print("With K={}".format(k))
    print("Silhouette with squared euclidean distance = " + str(silhouette))
    print('--'*30)

In [ ]:
model3.transform(cluster_final_data).groupBy('prediction').count().show()

In [ ]:
model2.transform(cluster_final_data).groupBy('prediction').count().show()

### Visualisations

In [ ]:
viz_cols = feat_cols + ['prediction']

pdf3 = predictions3.select(viz_cols).toPandas()
pdf2 = predictions2.select(viz_cols).toPandas()

In [ ]:
centers3_df = pd.DataFrame(model3.clusterCenters(), columns=feat_cols)
centers2_df = pd.DataFrame(model2.clusterCenters(), columns=feat_cols)

In [ ]:
def plot_clusters(data, centers, k, feature):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=feature, y='count', hue='prediction', data=data, palette='viridis')
    plt.scatter(centers[feature], centers['count'], s=300, c='red', marker='X')
    plt.title(f'K-means Clustering (k={k}) - {feature} vs Count')
    plt.xlabel(feature.capitalize())
    plt.ylabel('Count')
    plt.show()

In [ ]:
# Plot clusters for k=3
for feature in feat_cols:
    if feature != 'count':
        plot_clusters(pdf3, centers3_df, 3, feature)


In [ ]:
# Plot clusters for k=2
for feature in feat_cols:
    if feature != 'count':
        plot_clusters(pdf2, centers2_df, 2, feature)

#### Python Sklearn Compare 

In [ ]:
import random 
import numpy as np 
import matplotlib.pyplot as plt 
import sklearn
from sklearn.cluster import KMeans 
%matplotlib inline

In [ ]:
df_cl = df.toPandas()

df_cl.info()
df_cl.head()

In [ ]:
#Normalizing over the standard deviation
from sklearn.preprocessing import StandardScaler
X = df_cl.values[:, 1:]

X = np.nan_to_num(X)

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)


In [ ]:
clusterNum = 3
k_means = KMeans(init="k-means++", n_clusters=clusterNum, n_init=12)
k_means.fit(X_normalized)
labels = k_means.labels_
print(labels)

In [ ]:
df_cl["Cluster"] = labels

df_cl.head()

In [ ]:
df_cl.groupby('Cluster').mean()

In [ ]:
df_cl['Cluster'].value_counts()

In [ ]:
sns.pairplot(df_cl, hue='Cluster')

In [ ]:
sns.scatterplot(data=df_cl, x="count", y='temp',hue="Cluster")

### Linear Regression
#### PySpark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Initialize Spark session
spark = SparkSession.builder.appName('lr_example').getOrCreate()

data = df

# Show the data
data.show()

# Print the schema of the DataFrame
data.printSchema()

In [ ]:
data.columns

In [ ]:
# Define input columns and assembler
input_cols = ["season", "temp", "feels_temp", "windspeed", "casual", "registered", "Year", "Month", "Hour"]
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")


output = assembler.transform(data)
output.select("features").show(3)

output.head(1)

In [ ]:
final_data = output.select("features", "count")

In [ ]:
# Print schema and show final data
final_data.printSchema()
final_data.show()

### Train Model

In [ ]:
#Split data to training and test: 

train, test = final_data.randomSplit([0.7,0.3])

train.describe().show()
test.describe().show()

In [ ]:
lr = LinearRegression(labelCol='count')

lrModel = lr.fit(train)

In [ ]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients)))
print("Intercept: {}".format(str(lrModel.intercept)))

In [ ]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
trainingSummary.residuals.show()

In [ ]:
# Evaluate the model on the test data
test_results = lrModel.evaluate(test)
test_results.residuals.show()

In [ ]:
# Print RMSE and R2 values
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("R2: {}".format(test_results.r2))

In [ ]:
# Describe the final data
final_data.describe().show()

In [ ]:
#Cross-Validation
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

# Define the linear regression model
lr = LinearRegression(labelCol='count')

# Create a parameter grid for cross-validation (if needed)
paramGrid = ParamGridBuilder().build()

# Define the cross-validator
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol='count'),
                          numFolds=5)  # Use 5-fold cross-validation

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(final_data)

# Get the best model from cross-validation
bestModel = cvModel.bestModel

# Evaluate the best model on the test data
test_results_cv = bestModel.evaluate(test)
print("Cross-Validated RMSE: {}".format(test_results_cv.rootMeanSquaredError))
print("Cross-Validated R2: {}".format(test_results_cv.r2))



In [ ]:
#Inspect Model Results

import matplotlib.pyplot as plt
import seaborn as sns

# Make predictions on the test set
predictions = bestModel.transform(test)

# Collect predicted and actual values to the local environment for plotting
predictions_pd = predictions.select("prediction", "count").toPandas()

# Plot predicted vs. actual
plt.figure(figsize=(10, 6))
sns.scatterplot(x=predictions_pd['count'], y=predictions_pd['prediction'])
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Predicted vs Actual Values')
plt.plot([predictions_pd['count'].min(), predictions_pd['count'].max()],
         [predictions_pd['count'].min(), predictions_pd['count'].max()],
         color='red', linestyle='--')  # Line y=x for reference
plt.show()

# Residuals Plot
residuals = predictions_pd['count'] - predictions_pd['prediction']
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True, bins=30)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Residuals Distribution')
plt.show()

# Residuals vs. Fitted Values Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=predictions_pd['prediction'], y=residuals)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.axhline(0, color='red', linestyle='--')
plt.show()

# Q-Q Plot of Residuals
import numpy as np
import statsmodels.api as sm

plt.figure(figsize=(10, 6))
sm.qqplot(np.array(residuals), line='45')
plt.title('Q-Q Plot of Residuals')
plt.show()


# 08 - Interpretation


#### K-means Clustering

#### Linear Regression

Iteration